In [1]:
import pandas as pd
import psycopg2
import numpy as np

In [2]:
class DataPreparation:

    def __init__(self,path_to_books , path_to_users , path_to_rating):
        self.path_books = path_to_books
        self.path_users = path_to_users
        self.path_ratings = path_to_rating

        self.books = pd.read_csv(self.path_books, sep=";", on_bad_lines='skip', encoding="latin-1")
        self.books.columns = ['ISBN', 'bookTitle', 'bookAuthor', 'yearOfPublication', 'publisher',  'imageUrlS', 'imageUrlM', 'imageUrlL']
        self.users = pd.read_csv(self.path_users, sep=';', on_bad_lines='skip', encoding="latin-1")
        self.users.columns = ['userID', 'Location', 'Age']
        self.ratings = pd.read_csv(self.path_ratings, sep=';', on_bad_lines='skip', encoding="latin-1")
        self.ratings.columns = ['userID', 'ISBN', 'bookRating']
        
        #dropping bad columns 
        self.books = self.books[(self.books.yearOfPublication != 'DK Publishing Inc') & (self.books.yearOfPublication != 'Gallimard')]

        #type casting the column
        self.books.yearOfPublication = self.books.yearOfPublication.astype('int32')

        #droppping values with none as values
        self.books = self.books.dropna(subset=['publisher'])
          
        self.users.loc[(self.users.Age > 90) | (self.users.Age < 5), 'Age'] = np.nan
        self.users.Age = self.users.Age.fillna(self.users.Age.mean())
        self.users.Age = self.users.Age.astype(np.int32)

        self.ratings_new = self.ratings[self.ratings.ISBN.isin(self.books.ISBN)]

        #Hence segragating implicit and explict ratings datasets
        self.ratings_explicit = self.ratings_new[self.ratings_new.bookRating != 0]
        self.ratings_implicit = self.ratings_new[self.ratings_new.bookRating == 0]
        print(self.ratings_explicit.shape)

In [3]:
class DataPreparationPostgres(DataPreparation):
    def __init__(self, path_to_books, path_to_users, path_to_rating):
        super().__init__(path_to_books, path_to_users, path_to_rating)

        # PostgreSQL connection details
        self.db_host = 'localhost'
        self.db_port = '5432'
        self.db_name = 'library'
        self.db_user = 'postgres'
        self.db_password = 'admin'

    def store_data_in_postgres(self):
        # Connect to PostgreSQL
        conn = psycopg2.connect(
            host=self.db_host,
            port=self.db_port,
            database=self.db_name,
            user=self.db_user,
            password=self.db_password
        )

        # Create tables for books, users, and explicit ratings
        self.create_books_table(conn)
        self.create_users_table(conn)
        self.create_explicit_ratings_table(conn)

        # Insert data into the tables
        self.insert_books_data(conn)
        self.insert_users_data(conn)
        self.insert_explicit_ratings_data(conn)

        # Close the database connection
        conn.close()

    def create_books_table(self, conn):
        # Create the books table in PostgreSQL
        with conn.cursor() as cur:
            cur.execute('''
                CREATE TABLE IF NOT EXISTS books (
                    ISBN VARCHAR(20) PRIMARY KEY,
                    bookTitle VARCHAR(500),
                    bookAuthor VARCHAR(500),
                    yearOfPublication INT,
                    publisher VARCHAR(500),
                    imageUrlS VARCHAR(500),
                    imageUrlM VARCHAR(500),
                    imageUrlL VARCHAR(500)
                )
            ''')
            conn.commit()

    def create_users_table(self, conn):
        # Create the users table in PostgreSQL
        with conn.cursor() as cur:
            cur.execute('''
                CREATE TABLE IF NOT EXISTS users (
                    userID INT PRIMARY KEY,
                    Location VARCHAR(500),
                    Age INT
                )
            ''')
            conn.commit()

    def create_explicit_ratings_table(self, conn):
        # Create the explicit_ratings table in PostgreSQL
        with conn.cursor() as cur:
            cur.execute('''
                CREATE TABLE IF NOT EXISTS explicit_ratings (
                    userID INT,
                    ISBN VARCHAR(20),
                    bookRating INT,
                    PRIMARY KEY (userID, ISBN),
                    FOREIGN KEY (userID) REFERENCES users (userID),
                    FOREIGN KEY (ISBN) REFERENCES books (ISBN)
                )
            ''')
            conn.commit()

    def insert_books_data(self, conn):
        # Insert books data into the books table
        with conn.cursor() as cur:
            for _, book in self.books.iterrows():
                cur.execute('''
                    INSERT INTO books (ISBN, bookTitle, bookAuthor, yearOfPublication, publisher, imageUrlS, imageUrlM, imageUrlL)
                    VALUES (%s, %s, %s, %s, %s, %s, %s, %s)
                ''', (book['ISBN'], book['bookTitle'], book['bookAuthor'], book['yearOfPublication'],
                      book['publisher'], book['imageUrlS'], book['imageUrlM'], book['imageUrlL']))
            conn.commit()

    def insert_users_data(self, conn):
        # Insert users data into the users table
        with conn.cursor() as cur:
            for _, user in self.users.iterrows():
                cur.execute('''
                    INSERT INTO users (userID, Location, Age)
                    VALUES (%s, %s, %s)
                ''', (user['userID'], user['Location'], user['Age']))

    def insert_explicit_ratings_data(self, conn):
        # Insert explicit ratings data into the explicit_ratings table
        with conn.cursor() as cur:
            for _, rating in self.ratings_explicit.iterrows():
                cur.execute('''
                    INSERT INTO explicit_ratings (userID, ISBN, bookRating)
                    VALUES (%s, %s, %s)
                ''', (rating['userID'], rating['ISBN'], rating['bookRating']))
            conn.commit()



In [5]:
input = DataPreparationPostgres('../books_data/books.csv','../books_data/users.csv','../books_data/ratings.csv')

C:\Users\LENOVO\AppData\Local\Temp\ipykernel_14108\2570813239.py:8: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  self.books = pd.read_csv(self.path_books, sep=";", on_bad_lines='skip', encoding="latin-1")


(383839, 3)


In [6]:
input.store_data_in_postgres()